# Forecasting Macroeconomic variables with Neural Networks

## Abstract

> Econometrics has long been the lead forecasting method for many macro and micro variables and problems. Here we will use neural networks to see if they provide any advantage over the traditional use of econometrics and regression analysis. Preprocessing and making sure that we tune the parameters properly will play a big factor in whether our neural network can produce superior results over traditional econometrics. In this experiment we will create a neural network and compare it to some econometrics models to see if it's performance can surpass it.



## Variables

<br>
**CPI** *(Annual growth rate %)*

Inflation measured by consumer price index (CPI) is defined as the change in the prices of a basket of goods and services that are typically purchased by specific groups of households. Inflation is measured in terms of the annual growth rate and in index, 2010 base year with a breakdown for food, energy and total excluding food and energy. Inflation measures the erosion of living standards. A consumer price index is estimated as a series of summary measures of the period-to-period proportional change in the prices of a fixed set of consumer goods and services of constant quantity and characteristics, acquired, used or paid for by the reference population. Each summary measure is constructed as a weighted average of a large number of elementary aggregate indices. Each of the elementary aggregate indices is estimated using a sample of prices for a defined set of goods and services obtained in, or by residents of, a specific region from a given set of outlets or other sources of consumption goods and services.

**GDP** *(Percent Change same period previous year)*

Gross Domestic Product (GDP) is the standard measure of the value of final goods and services produced by a country during a period minus the value of imports. "Gross" signifies that no deduction has been made for the depreciation of machinery, buildings and other capital products used in production. "Domestic" means that it relates to the output produced on the economic territory of the country. The "products" refer to final goods and services, that is, those that are purchased, imputed or otherwise, as: the final consumption of households, non-profit institutions serving households and government; fixed capital formation; and exports (minus imports). GDP should not be looked upon as an all-encompassing measure for societies' well-being, as it does not include several aspects of people's material living standards let alone other aspects of people's quality of life. Data are internationally comparable by following the System of National Accounts. This indicator is seasonally adjusted and it is measured in percentage change from previous quarter and from same quarter previous year and based on real GDP.

**UR** *(Percent of labor force)*

Unemployment rate is the number of unemployed people as a percentage of the labour force, where the latter consists of the unemployed plus those in paid or self-employment. Unemployed people are those who report that they are without work, that they are available for work and that they have taken active steps to find work in the last four weeks. When unemployment is high, some people become discouraged and stop looking for work; they are then excluded from the labour force. This implies that the unemployment rate may fall, or stop rising, even though there has been no underlying improvement in the labour market.

**IR** - *Interest Rate (Immediate rates (< 24 hrs) Federal funds rate Percentage)*

Short-term interest rates are the rates at which short-term borrowings are effected between financial institutions or the rate at which short-term government paper is issued or traded in the market. Short-term interest rates are generally averages of daily rates, measured as a percentage. Short-term interest rates are based on three-month money market rates where available. Typical standardised names are "money market rate" and "treasury bill rate".

**LR10** - *Long-term government bond yields ()*



**CPI** *Growth rate same period previous year Percentage.*

Measures the price change in a basket for each item and averages them.

**GDP** *Growth rate same period previous year, s.a. Percentage*

Total value of goods produced and also the services provided during one year, hence the name "gross domestic product".

## Our Model

In [26]:
# load our functions and objects
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [27]:
# load data
macro = pd.read_csv("us_macro_var.csv")
# display data 
macro.head()

,TIME,CPI,GDP,UR,IR,LR10,LR10-IR,EXRATE
0,1998-Q1,1.462294,4.623460,4.633333,NaN,NaN,NaN,NaN
1,1998-Q2,1.602164,4.067661,4.400000,NaN,NaN,NaN,NaN
2,1998-Q3,1.595855,4.104639,4.533333,NaN,NaN,NaN,NaN
3,1998-Q4,1.548307,4.998974,4.433333,NaN,NaN,NaN,NaN
4,1999-Q1,1.667696,4.800728,4.300000,4.733333,0.25,4.983333,1.122667


In [28]:
# replace NA with median value
macro = macro.fillna(macro.median())
macro.head()
# turn time into row title instead of a variable
macro = macro.set_index('TIME')
macro.head(20)

,CPI,GDP,UR,IR,LR10,LR10-IR,EXRATE
TIME,,,,,,,
1998-Q1,1.462294,4.623460,4.633333,1.006667,1.901667,3.790000,1.251800
1998-Q2,1.602164,4.067661,4.400000,1.006667,1.901667,3.790000,1.251800
1998-Q3,1.595855,4.104639,4.533333,1.006667,1.901667,3.790000,1.251800
1998-Q4,1.548307,4.998974,4.433333,1.006667,1.901667,3.790000,1.251800
1999-Q1,1.667696,4.800728,4.300000,4.733333,0.250000,4.983333,1.122667
1999-Q2,2.109359,4.649637,4.266667,4.746667,0.793333,5.540000,1.056933
1999-Q3,2.345981,4.598207,4.233333,5.093333,0.790000,5.883333,1.049067
1999-Q4,2.622484,4.694708,4.066667,5.306667,0.833333,6.140000,1.038133
2000-Q1,3.240178,4.166983,4.033333,5.676667,0.803333,6.480000,0.986933


In [31]:
macro1 = macro.values
# split into x and y variables
X = macro1[:,0:6]
Y = macro1[:,6]

In [43]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(3, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # compile
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [44]:
# random seed to reproduce NN
seed = 7
np.random.seed(seed)

In [45]:
# evaluation of model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
# eval with 10 fold cross validation
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)

print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -0.06 (0.05) MSE
